In [1]:
from datetime import datetime, timedelta
from collections import defaultdict

In [7]:
def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")
data = [
    {
        "item_code": "Executive Desk (4*2)",
        "rate": "1080.0",
        "qty": "5",
        "start_date": "2024-11-10",
        "stop_date": "2024-11-30"
    },
    {
        "item_code": "Executive Desk (4*2)",
        "rate": "1080.0",
        "qty": "10",
        "start_date": "2024-11-01",
        "stop_date": "2024-11-30"
    },
    {
        "item_code": "Chair",
        "rate": "200.0",
        "qty": "15",
        "start_date": "2024-10-20",
        "stop_date": "2024-11-15"
    }
]
def generate_monthly_bill(data, target_month):
    target_start = parse_date(f"{target_month}-01")
    next_month = target_start.replace(day=28) + timedelta(days=4)
    target_end = next_month - timedelta(days=next_month.day)
    days_in_month = (target_end - target_start).days + 1

    grouped = defaultdict(lambda: {'qty': 0, 'amount': 0.0})

    for entry in data:
        item_code = entry['item_code']
        rate = float(entry['rate'])
        qty = int(entry['qty'])
        start_date = parse_date(entry['start_date'])
        stop_date = parse_date(entry['stop_date'])

        billing_start = max(start_date, target_start)
        billing_end = min(stop_date, target_end)

        if billing_start <= billing_end:
            active_days = (billing_end - billing_start).days + 1
            daily_rate = (rate * qty) / days_in_month
            amount = round(daily_rate * active_days, 2)

            billing_period = f"{billing_start.date()} to {billing_end.date()}"
            key = (item_code, rate, billing_period)

            grouped[key]['qty'] += qty
            grouped[key]['amount'] += amount

    line_items, total_revenue = [], 0.0

    for (item_code, rate, billing_period), values in grouped.items():
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values['qty'],
            "amount": round(values['amount'], 2),
            "billing_period": billing_period
        })
        total_revenue += values['amount']

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }
target_month = "2024-11"
result = generate_monthly_bill(data, target_month)
print(" Monthly Billing Summary:")
for item in result["line_items"]:
    print(item)

print("\n Total Revenue:", result["total_revenue"])


 Monthly Billing Summary:
{'item_code': 'Executive Desk (4*2)', 'rate': 1080.0, 'qty': 5, 'amount': 3780.0, 'billing_period': '2024-11-10 to 2024-11-30'}
{'item_code': 'Executive Desk (4*2)', 'rate': 1080.0, 'qty': 10, 'amount': 10800.0, 'billing_period': '2024-11-01 to 2024-11-30'}
{'item_code': 'Chair', 'rate': 200.0, 'qty': 15, 'amount': 1500.0, 'billing_period': '2024-11-01 to 2024-11-15'}

 Total Revenue: 16080.0
